In [235]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [236]:
#Read the data

X = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col ='Id')
X_test_full = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col ='Id')

X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [237]:
X.shape

(1460, 80)

There are a total of 1460 rows and 80 columns (features )

In [238]:
#drop rows with missing targets
X_with_na = X['SalePrice'].isna().sum()
print(X_with_na)

0


There are no rows with missing target

In [239]:
#Seperate predictor from target
y = X.SalePrice
y.shape

(1460,)

In [240]:
# X_full.shape -> 80 columns still remaining in original data

# Remove target from predictor
#inplace = True modifies original given data, inplace = False uses to store modified data into another variable
X.drop(['SalePrice'], axis=1, inplace=True)
X.shape

(1460, 79)

In [241]:
# split training and validation data
from sklearn.model_selection import train_test_split

X_train_full, X_valid_full , y_train , y_valid = train_test_split(X,y,
                                                                 train_size = 0.8, test_size=0.2, random_state=0)

In [242]:
#select columns with low cardinality(less unique values) and catergorical values (object dtype)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique()<10 and
                       X_train_full[cname].dtype == 'object']

#select numberical columns

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

my_cols = low_cardinality_cols + numerical_cols

len(my_cols)

76

In [243]:
X_train_full[numerical_cols].isna().sum()

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

76 colums will be used ( 4 will be dropped )

In [244]:
# use my_cols only

X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [245]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy = 'mean')
#The following returns Numpy array
X_train_imputed = imputer.fit_transform(X_train[numerical_cols])
X_valid_imputed= imputer.transform(X_valid[numerical_cols])

# Convert Numpy array back to DataFrame
X_train_imputed = pd.DataFrame(X_train_imputed, columns=numerical_cols, index=X_train_full.index)
X_valid_imputed = pd.DataFrame(X_valid_imputed, columns=numerical_cols, index=X_valid_full.index)

In [246]:
# X_valid_imputed.isna().sum()

In [247]:
#One hot encode using pandas

X_train_imputed = pd.get_dummies(X_train_imputed)
X_valid_imputed = pd.get_dummies(X_valid_imputed)
X_test = pd.get_dummies(X_test)

#X_train_imputed.shape = (1168, 282)
#X_valid_imputed.shape = (292, 256)
#X_test.shape = (1459, 282)

#After one hot encoding , shapes of train,valid,test may not be equal.
#Therefore aligning has to be done

X_train_imputed, X_valid_imputed = X_train_imputed.align(X_valid_imputed, join='left', axis=1)
X_train_imputed , X_test = X_train_imputed.align(X_test, join='left', axis=1)

#After imputation , the shape is as follows
#X_train_imputed.shape = (1168, 282)
#X_valid_imputed.shape = (292, 282)
#X_test.shape = (1459, 282)

In [248]:
#Build a model
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators = 1000, learning_rate =0.03 ,n_jobs=4, max_depth=5)
my_model.fit(X_train_imputed,y_train,
            early_stopping_rounds =5,
            eval_set =[(X_valid_imputed,y_valid)],
            verbose = False )

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

In [249]:
#Get predictions

predictions = my_model.predict(X_valid_imputed)

In [250]:
from sklearn.metrics import mean_squared_error

log_y_valid = np.log(y_valid)
log_predictions = np.log(predictions)


rmse = mean_squared_error(log_y_valid, log_predictions, squared=False)
print(rmse)

0.13541427485489368


In [251]:
preds_test = my_model.predict(X_test)

In [233]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})

output.to_csv('submission.csv', index=False)